# Create PCAPs with specific patterns and payload sizes for measurement

In [39]:
from scapy.all import Ether, Dot1Q, RawPcapWriter, IP, UDP

In [42]:
ETH_DST = '1c:1b:0d:9a:70:8c'
DST_IP = '192.168.1.104'
VLAN_ID = 123
VLAN_TYPE = 0x0800

In [41]:
def get_header(payload, pkt_id, vlan_prio, eth_src, src_ip):
    return Ether(dst=ETH_DST,
                     src=eth_src
                    )/Dot1Q(prio=vlan_prio,
                            vlan=VLAN_ID,
                            type=VLAN_TYPE
                           )/IP(id=pkt_id,
                                version=4,
                                dst=DST_IP,
                                src=src_ip)/UDP(sport=10001,
                                                    dport=10002)/payload

## Video Use Case
- mean frame size from video_trace2 = 3527B
- send each 1/24s 3527B

In [45]:
FILE_VIDEO = 'video.pcap'
ETH_SRC = '00:00:00:00:00:03'
SRC_IP = '192.168.1.222'
PAYLOAD_FULL = 1472 * '0'
PAYLOAD_REST = 583 * '0'
VLAN_PRIO = 3

pcap = RawPcapWriter(open(FILE_VIDEO, 'wb'), nano=True, linktype=1, sync=True, append=True)

ID = 1

for i in range(0,60):
    for u in range(0,24):
        usec = u*1/24
        
        usec_round = int(round(usec,9) * 1e9)
        header = get_header(PAYLOAD_FULL, ID, VLAN_PRIO, ETH_SRC, SRC_IP)
        pcap.write_header(header)
        pcap.write_packet(header, sec=i, usec=usec_round)
        ID += 1
        
        usec_round = int(round(usec + 0.000000001,9) * 1e9)
        header = get_header(PAYLOAD_FULL, ID, VLAN_PRIO, ETH_SRC, SRC_IP)
        pcap.write_header(header)
        pcap.write_packet(header, sec=i, usec=usec_round)
        ID += 1
        
        usec_round = int(round(usec + 0.000000002,9) * 1e9)
        header = get_header(PAYLOAD_REST, ID, VLAN_PRIO, ETH_SRC, SRC_IP)
        pcap.write_header(header)
        pcap.write_packet(header, sec=i, usec=usec_round)
        ID += 1

## Robotic Use Case
- send each ms 128B incl. header

In [46]:
FILE_ROBOTIC = 'robotic.pcap'
ETH_SRC = '00:00:00:00:00:01'
SRC_IP = '192.168.1.238'
PAYLOAD = 82 * '0'
VLAN_PRIO = 1

pcap = RawPcapWriter(open(FILE_ROBOTIC, 'wb'), nano=True, linktype=1, sync=True, append=True)

ID = 1

for sec in range(0,60):
    for usec in range(0,1000):
        usec = usec * 1000000
        header = get_header(PAYLOAD, ID, VLAN_PRIO, ETH_SRC, SRC_IP)
        pcap.write_header(header)
        pcap.write_packet(header, sec=sec, usec=usec)
        ID += 1

## Audio Use Case
- send each 20ms 85B + Header

In [47]:
FILE_AUDIO = 'audio.pcap'
ETH_SRC = '00:00:00:00:00:02'
SRC_IP = '192.168.1.222'
PAYLOAD = 85 * '0'
VLAN_PRIO = 2

pcap = RawPcapWriter(open(FILE_AUDIO, 'wb'), nano=True, linktype=1, sync=True, append=True)
pcap.write_header(pkt)

ID = 1

for sec in range(0,60):
    for usec in range(0,50):
        usec = usec * 20000000
        header = get_header(PAYLOAD, ID, VLAN_PRIO, ETH_SRC, SRC_IP)
        pcap.write_header(header)
        pcap.write_packet(header, sec=sec, usec=usec)
        ID += 1